<a href="https://colab.research.google.com/github/fjme95/aplicaciones-financieras/blob/main/Modulo%203/Semana%201/Manejo_de_portafolios_Markovitz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

# Dependencias

In [1]:
!pip install yfinance -U plotly

     |████████████████████████████████| 27.7 MB 5.3 MB/s 
     |████████████████████████████████| 6.4 MB 36.9 MB/s 
     |████████████████████████████████| 63 kB 860 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import yfinance as yf

import pandas as pd
import numpy as np

import cvxopt as opt

import plotly.express as px

# Datos


Los datos con los que trabajaremos son el precio de acciones de Google (GOOG), Apple (AAPL), IBM (IBM), Microsoft (MSFT) y ExxonMobil (XOM) del último año. 

Para obtener los datos, usaremos [```yfinance```](https://github.com/ranaroussi/yfinance) y nos centraremos en el precio de cierre ajustado (Adj Close).

In [3]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "GOOG AAPL IBM MSFT XOM",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "1y",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        # group_by = 'ticker',
    ).loc[:, 'Adj Close']
data

[*********************100%***********************]  5 of 5 completed


,AAPL,GOOG,IBM,MSFT,XOM
Date,,,,,
2021-02-11,134.331619,2095.889893,111.526123,242.457657,47.226631
2021-02-12,134.570175,2104.110107,111.424660,242.953491,47.870979
2021-02-16,132.403061,2121.899902,110.751312,241.674210,49.311275
2021-02-17,130.066940,2128.310059,110.659081,242.727814,50.078800
2021-02-18,128.943634,2117.199951,111.360085,242.320282,49.292328
...,...,...,...,...,...
2022-02-04,172.389999,2860.320068,137.149994,305.940002,80.517395
2022-02-07,171.660004,2778.760010,137.240005,300.950012,81.486641
2022-02-08,174.830002,2784.260010,137.020004,304.559998,79.379997


Obtenemos los retornos logarítmicos anualizados para cada activo.

In [4]:
annual_returns = np.log(data / data.shift()) / (1 / 252) # Para anualizar los retornos
# annual_returns

mean_returns = annual_returns.mean()
# mean_returns

cov_returns = annual_returns.cov()
cov_returns

,AAPL,GOOG,IBM,MSFT,XOM
AAPL,15.541453,9.117077,1.337586,9.638895,1.239742
GOOG,9.117077,14.215318,1.014269,9.885287,3.176758
IBM,1.337586,1.014269,11.508228,0.050778,5.676060
MSFT,9.638895,9.885287,0.050778,12.286223,0.171504
XOM,1.239742,3.176758,5.676060,0.171504,20.515706


# Markovitz

https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html

> Si $w$ es el vector de pesos de las acciones con retornos esperados $\mu$, entonces el retorno del portafolio es igual al peso de cada acción multiplicado por su retorno, es decir, $w^T \mu$. El riesgo del portafolio en términos de la matriz de covarianzas $\Sigma$ esta dado por $\sigma^2 = w^T \Sigma w$. 

> La razón de Sharpe es el retorno en exceso del portafolio por unidad de riesgo (volatilidad)

$$
SR = \frac{R_p-R_f}{\sigma}
$$

Con esto en mente, crearemos un cantidad $N$ de portafolios con pesos aleatorios y guardaremos tanto los pesos, como el retorno, la volatilidad y la razon de Sharpe para cada uno.

In [5]:
np.random.seed(1995)

N = 10000
k = annual_returns.shape[1]

weights = np.zeros((N, k))
returns = np.zeros(N)
volatilities = np.zeros(N)
sharpe_ratios = np.zeros(N)

for i in range(N):
    w = np.random.random(k)
    w /= np.sum(w)

    weights[i, :] = w

    returns[i] = np.dot(mean_returns, w)

    volatilities[i] = np.sqrt(np.dot(w.T, np.dot(cov_returns, w))) # w.T @ cov_returns @ w

    sharpe_ratios[i] = returns[i] / volatilities[i]

Graficamos la volatilidad contra el retorno de cada portafolio generado y coloreamos en función de la razón de Sharpe. Los portafolios (casi) óptimos, serían aquellos que tienen el mayor retorno para cierto nivel de volatilidad, o dicho de otra manera, los que tienen la menor volatilidad para algún retorno especificado.

In [6]:
import matplotlib.pyplot as plt

px.scatter(x = volatilities, y = returns, color = sharpe_ratios,
           labels={
                     "x": "Volatilidad",
                     "y": "Retorno",
                     "color": "Razón de Sharpe"
                 }
           )
# plt.scatter(volatilities, returns, c = sharpe_ratios)

> La optimización del portafolio se puede ver como un problema de optimización convexa y una solución puede encontrarse usando programación cuadrática. Si denotamos el retorno objetivo como $\mu^*$, el problema a resolver para el portafolio sólo con posiciones largar es:

\begin{align}
    \text{min}_w && w^T \Sigma w \\
    \text{s.a.} && w^t \mu \geq \mu^*  \\
    && w^T \mathbf{1} = 1 \\
    && w_i \geq 0
\end{align}

Para resolverlo, ocuparemos la función [```covxopt.solvers.qp```](https://cvxopt.org/userguide/coneprog.html#quadratic-programming). Esta requiere que el problema de optimización se encuentre en la forma general. A saber, la forma general de un problema de programación cuadrática es la siguiente:

\begin{align}
    \text{min}_x && \frac{1}{2}x^TPx + q^Tx \\
    \text{s.a.} && Gx \preceq h \\
    && Ax = b
\end{align}

In [7]:
mu_star = .35

In [8]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

In [9]:
results = opt.solvers.qp(P, q, G, h, A, b)
results

     pcost       dcost       gap    pres   dres
 0:  5.4833e+00  4.5410e+00  9e+00  3e+00  4e+00
 1:  5.5044e+00  4.8928e+00  1e+00  2e-01  4e-01
 2:  6.9802e+00  6.8958e+00  2e+00  1e-01  2e-01
 3:  7.3411e+00  7.3041e+00  8e-02  3e-03  4e-03
 4:  7.3605e+00  7.3592e+00  2e-03  3e-05  5e-05
 5:  7.3608e+00  7.3607e+00  2e-05  3e-07  5e-07
 6:  7.3608e+00  7.3608e+00  2e-07  3e-09  5e-09
Optimal solution found.


{'dual infeasibility': 4.629932898969916e-09,
 'dual objective': 7.360762834925122,
 'dual slack': 3.2290478647743733e-09,
 'gap': 1.8014361052520309e-07,
 'iterations': 6,
 'primal infeasibility': 3.1136636573876118e-09,
 'primal objective': 7.360762967281933,
 'primal slack': 6.091993319537766e-10,
 'relative gap': 2.447349745741884e-08,
 's': <6x1 matrix, tc='d'>,
 'status': 'optimal',
 'x': <5x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 'z': <6x1 matrix, tc='d'>}

Los pesos del portafolio óptimo los obtenemos de la llave ```x```

In [10]:
w = np.asarray(results['x']).reshape((-1))
np.dot(mean_returns, w)

0.3499999993380514

La volatilidad de ```primal objective```

In [11]:
np.sqrt(results['primal objective'])

2.7130726063417345

Si variamos los retornos objetivos, podemos obtener la **Frontera Eficiente**, que esta constituida de los portafolios óptimos para distintos niveles del retorno.

In [12]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

mu_stars = np.linspace(.25, .45, 20)

ws = np.zeros((len(mu_stars), k))
mus = np.zeros(len(mu_stars))
sigmas = np.zeros(len(mu_stars))

for i, mu_star in enumerate(mu_stars):
    try:
        h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
        results = opt.solvers.qp(P, q, G, h, A, b)

        w = np.asarray(results['x']).reshape((-1))
        ws[i, :] = w
        mus[i] = np.dot(mean_returns, w)
        sigmas[i] = np.sqrt(results['primal objective'])
    except:
        print('domain error')

     pcost       dcost       gap    pres   dres
 0:  5.4831e+00  4.3930e+00  1e+00  1e-17  4e+00
 1:  5.4790e+00  5.4486e+00  3e-02  2e-16  1e-01
 2:  5.4778e+00  5.4769e+00  9e-04  1e-16  1e-03
 3:  5.4777e+00  5.4777e+00  2e-05  6e-17  1e-05
 4:  5.4777e+00  5.4777e+00  9e-07  1e-16  1e-07
 5:  5.4777e+00  5.4777e+00  1e-08  6e-17  1e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  5.4831e+00  4.4076e+00  8e+00  2e+00  4e+00
 1:  5.5048e+00  4.7885e+00  8e-01  3e-02  5e-02
 2:  5.4953e+00  5.3351e+00  2e-01  6e-03  1e-02
 3:  5.5012e+00  5.4920e+00  9e-03  1e-16  2e-14
 4:  5.4992e+00  5.4990e+00  2e-04  6e-17  2e-15
 5:  5.4992e+00  5.4992e+00  2e-06  2e-16  3e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  5.4831e+00  4.4225e+00  9e+00  2e+00  4e+00
 1:  5.5047e+00  4.7971e+00  8e-01  6e-02  9e-02
 2:  5.5483e+00  5.3879e+00  2e-01  7e-03  1e-02
 3:  5.5623e+00  5.5549e+00  8e-03  2e-04  2e-04
 4:  5.5623e+00  5.5621e

Finalmente, podemos graficar la frontera eficiente. Se puede observar que, pese a que las simulaciones nos dieron portafolios a la frontera, no eran realmente óptimos.

In [13]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_traces(
    [
        go.Scatter(
            x = volatilities, y = returns, 
            marker = dict(
                color = sharpe_ratios,
                colorbar = dict(title="Razón de Sharpe")
            ), 
            mode = 'markers', 
            showlegend = False
        ), 
        go.Scatter(
            x = sigmas, y = mus, 
            mode = 'lines + markers',  
            showlegend = False
        )
])

fig.update_layout(
    xaxis_title = 'Volatilidad',
    yaxis_title = 'Retorno'
)



Notas

1. Se están usando los retornos sin considerar la tasa libre de riesgo. Es decir, no se consideran los retornos en exceso (?)

Ejercicios

1. Encontrar y graficar el portafolio óptimo de acuerdo a la razón de Sharpee 
2. Agregar los puntos de los portafolios que contienen únicamente un activo
3. Encontrar el portafolio con la menor volatilidad
4. Restar la tasa libre de riesgo a los retornos

TODO

1. Mencionar PyPortfolioOpt

# Mean-Variance Choice

El portafolio óptimo también se podría obtener maximizando, respecto a $w$,

$$
U(\mu, \Sigma; w) = w^T\mu - \frac{\delta}{2}w^T\Sigma w
$$

donde $\delta > 0$ es el parámetro de aversión al riesgo. La condición de primer orden para maximizarla es

$$
\mu = \delta \Sigma w
$$

lo que implica el siguiente diseño para un portafolio con riesgo:

$$
w = \left( \delta \Sigma \right)^{-1} \mu
$$

Es un sistema de ecuaciones lineales que podemos resolver con [```np.linalg.solve```](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).

In [14]:
delta = .2

np.linalg.solve(delta * cov_returns,  mean_returns)

array([0.02526523, 0.01812711, 0.01738135, 0.05381869, 0.11344088])

# Ligas interesantes

1. https://python-advanced.quantecon.org/black_litterman.html